In [13]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding,Bidirectional, Dense, Dropout , GRU
from tensorflow.keras.models import Sequential
from keras.layers import BatchNormalization
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [7]:
# Load data from CSV
train = pd.read_csv("train.csv")
train

,comment,label
0,مهبل عاقلين حدك يخدم بلد سويت مادية,1
1,غض البصر عن المحرم ليس بي السهل عن الرجال والن...,1
2,يا لكم من جهلة تدعون العلم أين أنتم من البوصي...,1
3,انا مكبوت كرحلي الكبت نخم باش نسامحك وراس جبهت...,1
4,انهم يحلون في مابينهم واستنساخ ادعية تحريظية ف...,1
...,...,...
11315,ماتو حبيبتي وبقاو هاذو الحثالة ياو فاقو قع هذا...,0
11316,حضو لي حاكم ولاية و بلدية دو راهم يرسكو على جا...,1
11317,الصورة في الوراء من التبوريضة المغربية حتى هي ...,1
11318,اشمن تهريب الزطله انت مالها منين كاتجي من حداك...,0


In [8]:
test= pd.read_csv("test.csv")
test

,comment,label
0,قيل لأحد الصالحين؛ كم بينا وبينعرش الرحمن ؟ ...,0
1,شروق غلطت غلط كبير إعلان حول تواصل إجتماع نسيب...,0
2,عيب وعار أن الرئيس الجزائري يقول إن الصحراء مح...,1
3,على بلحاج راجل و الدولة عميلة فرنسا العدو كل ا...,1
4,لكن في المغرب و له الحمد يوم بعد يوم نتلمس الق...,0
...,...,...
2826,وعلى كل حال الرواية لا تعود لسلفية ولا تعبر عن...,0
2827,مراكش قادر على القطاع نتاعك دير الاستقالة نتاع...,0
2828,لاحول ولا قوة الاباله الهم إن هذا منكر وبدع وش...,0
2829,تمثيل صفر كفاه يحبس فالوطو وخاىج بجري قدامها,0


In [9]:
train_texts = train.comment.astype(str)
test_texts = test.comment.astype(str)

train_labels = train.label.astype(int)
test_labels = test.label.astype(int)


# Tokenization
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_texts)
max_words = len(tokenizer.word_index) + 1 
max_len = 100
sequences_train = tokenizer.texts_to_sequences(train_texts)
train_data = pad_sequences(sequences_train, maxlen=max_len)
sequences_test = tokenizer.texts_to_sequences(test_texts)
test_data = pad_sequences(sequences_test, maxlen=max_len)

# Train-test split
X_train = train_data
X_test = test_data
y_train = train_labels 
y_test = test_labels

In [10]:
# Load FastText word embeddings
embedding_dim = 300
embeddings_index = {}
with open( r"C:\Users\Lenovo\Documents\PFE_Master\CODE\cc.ar.300.vec", encoding='utf-8') as f: 
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

# Prepare embedding matrix
word_index = tokenizer.word_index
embedding_matrix = np.zeros((max_words, embedding_dim))
for word, i in word_index.items():
    if i < max_words:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

In [11]:
def plot_performance(history=None, figure_directory=None, ylim_pad=[0, 0]):
    xlabel = 'Epoch'
    legends = ['Training', 'Validation']

    plt.figure(figsize=(20, 5))

    y1 = history.history['accuracy']
    y2 = history.history['val_accuracy']

    min_y = min(min(y1), min(y2))-ylim_pad[0]
    max_y = max(max(y1), max(y2))+ylim_pad[0]


    plt.subplot(121)

    plt.plot(y1)
    plt.plot(y2)

    plt.title('Model Accuracy\n', fontsize=17)
    plt.xlabel(xlabel, fontsize=15)
    plt.ylabel('Accuracy', fontsize=15)
    plt.ylim(min_y, max_y)
    plt.legend(legends, loc='upper left')
    plt.grid()

    y1 = history.history['loss']
    y2 = history.history['val_loss']

    min_y = min(min(y1), min(y2))-ylim_pad[1]
    max_y = max(max(y1), max(y2))+ylim_pad[1]


    plt.subplot(122)

    plt.plot(y1)
    plt.plot(y2)

    plt.title('Model Loss\n', fontsize=17)
    plt.xlabel(xlabel, fontsize=15)
    plt.ylabel('Loss', fontsize=15)
    plt.ylim(min_y, max_y)
    plt.legend(legends, loc='upper left')
    plt.grid()
    if figure_directory:
        plt.savefig(figure_directory+"/history")

    plt.show()

In [15]:
# Build Bi-GRU model

model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=max_len, weights=[embedding_matrix], trainable=False))
model.add(Bidirectional(GRU(256, return_sequences=True)))
model.add(BatchNormalization())  
model.add(Dropout(0.5))
model.add(Bidirectional(GRU(256)))
model.add(BatchNormalization())  
model.add(Dense(1, activation='sigmoid'))
 
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)
# Train the model
hist = model.fit(X_train, y_train, epochs=2, batch_size=16, validation_data=(X_val, y_val))

# Evaluate the model
y_pred = (model.predict(X_test) > 0.5).astype("int32")
print(classification_report(y_test, y_pred))
plot_performance(history=hist)

